# Project 3: Colab Training Runner (GitHub + Drive)
**Pre-requisites:**
1. Switch Runtime to GPU (Runtime -> Change runtime type -> T4 GPU).
2. Upload `datasets.zip` to Google Drive folder `Project3`.
3. Have your code on GitHub (Public or Private with Token).

In [ ]:
import torch
if torch.cuda.is_available():
    print(f"✅ GPU is ready: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU NOT detected. Go to Runtime -> Change runtime type -> T4 GPU")

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
DRIVE_ROOT = "/content/drive/MyDrive/Project3"
os.makedirs(DRIVE_ROOT, exist_ok=True)

In [ ]:
# REPLACE THIS with your specific Repo URL
# e.g., https://<TOKEN>@github.com/User/Repo.git
REPO_URL = "https://github.com/GurYon19/project3.git" 
REPO_NAME = "project3"

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL}
else:
    print("Repo already cloned. Pulling latest changes...")
    %cd {REPO_NAME}
    !git pull
    %cd ..

In [ ]:
import zipfile
ZIP_PATH = os.path.join(DRIVE_ROOT, "datasets.zip")
DEST_DIR = f"/content/{REPO_NAME}"

if os.path.exists(ZIP_PATH):
    print(f"📦 Unzipping data to {DEST_DIR}...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(DEST_DIR)
    print("✅ Extraction complete!")
else:
    print(f"❌ datasets.zip not found in {DRIVE_ROOT}")

## 🔍 AUTO-DETECT DATA PATH

In [ ]:
# Find where the annotations actually are to set the correct flag
search_root = f"/content/{REPO_NAME}"
found_path = None

for root, dirs, files in os.walk(search_root):
    if "_annotations.coco.json" in files and "train" in root:
        parent = os.path.dirname(root)
        found_path = parent
        print(f"✅ Found dataset at: {found_path}")
        break

if not found_path:
    print("❌ Could not automatically find dataset path.")
    found_path = "datasets/part2"
else:
    try:
        found_path = os.path.relpath(found_path, start=f"/content/{REPO_NAME}")
    except:
        pass

In [ ]:
# Run training using the detected path
%cd /content/{REPO_NAME}
print(f"Starting training using data dir: {found_path}")

!python part2_train.py --data-dir "{found_path}" --subset 0.5 --batch-size 16

In [ ]:
# --------------------------------------------------------
# BACKUP CHECKPOINTS & LOGS TO DRIVE
# --------------------------------------------------------
import shutil
import time

# 1. Backup Checkpoints
dest_ckpt = os.path.join(DRIVE_ROOT, "checkpoints/part2")
os.makedirs(dest_ckpt, exist_ok=True)
if os.path.exists("checkpoints/part2"):
    shutil.copytree("checkpoints/part2", dest_ckpt, dirs_exist_ok=True)
    print(f"✅ Saved CHECKPOINTS to Drive: {dest_ckpt}")
else:
    print("⚠️ No checkpoints found locally to backup.")

# 2. Backup TensorBoard Logs
dest_logs = os.path.join(DRIVE_ROOT, "logs/part2")
os.makedirs(dest_logs, exist_ok=True)
if os.path.exists("logs/part2"):
    shutil.copytree("logs/part2", dest_logs, dirs_exist_ok=True)
    print(f"✅ Saved TENSORBOARD LOGS to Drive: {dest_logs}")
else:
    print("⚠️ No logs found locally to backup.")
    
print(f"\nSynced at {time.ctime()}")